In [1]:
import csv
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from datetime import datetime
import re

# TripAdvisor Script 1 Version 2
Features:
- Grab all reviews based on TripAdvisor language filtered amount

Bugs:
- Some attractions the amount of filtered reviews can be inaccurate.

In [2]:
def scrape_reviews_v2_s1(attraction, link): 
  
  print(f"[INFO] Currently Scrapping: {attraction}")
  
  start_time = datetime.now()
  
  file_path = str(datetime.now().strftime('%Y_%m_%d')) + '_' + attraction.replace(" ", "_").lower() + '.csv'
  file = open(f'{file_path}', 'a', encoding="utf-8", newline='')
  csv_writer = csv.writer(file)
  
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
  driver.get(link)

  # Get number of Bahasa Indonesia reviews
  # Using this number to get number of pages to loop. 5 reviews/page
  native_num = driver.find_element(By.XPATH, "//label[@for='LanguageFilter_3']/span[2]").text
  native_num = int(re.search(r'\d+', native_num).group())
  num_pages = native_num // 5 + 1

  # Total review count
  total_reviews = driver.find_element(By.XPATH, "//span[@class='reviewCount siNVx S4 H3 Ci']").text
  total_reviews = int(''.join(i for i in total_reviews if i.isdigit()))
  
  review_count = 0
  page_num = 1
  
  for i in range(num_pages-1):
    
    # Let page load (change to until element exist later)
    time.sleep(3) 
    
    # Expand the review
    driver.find_element(By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]").click()

    # Extract elements
    users = driver.find_elements(By.XPATH, "//div[@class='bcaHz']/span[1]/a[1]")
    reviews = driver.find_elements(By.XPATH, "//q[@class='XllAv H4 _a']")
    dates = driver.find_elements(By.XPATH, "//span[@class='euPKI _R Me S4 H3']")
    
    # Only take remaining amount of reviews on last page
    rem = native_num % 5
    
    for j in range(len(reviews)):
      user = users[j].text
      review = reviews[j].text
      date = dates[j].text
      csv_writer.writerow([date, user, review]) 
 
    review_count += len(reviews)
    
    # Click next page
    next = driver.find_element(By.XPATH, "//a[@class='ui_button nav next primary ']")
    next.click()
    page_num += 1
    
    print(f"Review Count: {review_count}")
    print(f"Current page: {page_num}")
    
  else:
    # Let page load (change to until element exist later)
    time.sleep(3) 
    
    # Expand the review
    driver.find_element(By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]").click()
    
    users = driver.find_elements(By.XPATH, "//div[@class='bcaHz']/span[1]/a[1]")
    reviews = driver.find_elements(By.XPATH, "//q[@class='XllAv H4 _a']")
    dates = driver.find_elements(By.XPATH, "//span[@class='euPKI _R Me S4 H3']")
    
    for k in range(rem):
      user = users[k].text
      review = reviews[k].text
      date = dates[k].text
      csv_writer.writerow([date, user, review]) 
 
    review_count += rem
    
    print(f"Review Count: {review_count}")
    print(f"Current page: {page_num}")
  
  print('-------------- Scrape Completed --------------')
  print(f"Total Reviews: {total_reviews}")
  print(f"Total Native Reviews: {review_count}")
  end_time = datetime.now()
  print('Runtime: {}'.format(end_time - start_time))

  file.close()
  driver.close()

In [3]:
scrape_reviews_v2_s1("cablecar", "https://www.tripadvisor.co.id/Attraction_Review-g294264-d1997266-Reviews-Singapore_Cable_Car_Sentosa-Sentosa_Island.html#REVIEWS")



====== WebDriver manager ======


[INFO] Currently Scrapping: cablecar


Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Luqman\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


Review Count: 5
Current page: 2
Review Count: 10
Current page: 3
Review Count: 15
Current page: 4
Review Count: 20
Current page: 5
Review Count: 25
Current page: 6
Review Count: 27
Current page: 6
-------------- Scrape Completed --------------
Total Reviews: 2186
Total Native Reviews: 27
Runtime: 0:00:24.117581
